In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('your_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

features = ['Open', 'High', 'Low', 'Close']

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])

scaled_df = pd.DataFrame(scaled_data, columns=features, index=df.index)


In [ ]:
def create_sequences(data, target_cols, seq_len=10):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i + seq_len])
        y.append(data[i + seq_len, target_cols])
    return np.array(X), np.array(y)

target_cols = [1, 2, 3]
seq_len = 60

X, y = create_sequences(scaled_data, target_cols, seq_len)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Input

In [ ]:
# LSTM model
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(50, return_sequences=True),
    Dropout(0.5),  # Dropout layer
    LSTM(50, return_sequences=False),
    Dropout(0.5),
    Dense(25, activation='relu'),
    Dense(3)  # Output layer: 3 predictions (High, Low, Close)
])


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,  # Adjust based on performance
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Evaluate performance
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

In [ ]:
# Inverse scale the test set targets
scaled_y_test = np.zeros((y_test.shape[0], scaled_data.shape[1]))
scaled_y_test[:, target_cols] = y_test
true_y_test = scaler.inverse_transform(scaled_y_test)[:, target_cols]

# Inverse scale the predictions
scaled_preds = np.zeros((y_test.shape[0], scaled_data.shape[1]))
scaled_preds[:, target_cols] = predictions
true_preds = scaler.inverse_transform(scaled_preds)[:, target_cols]

# Plot predictions vs actual values for High, Low, and Close
plt.figure(figsize=(18, 12))

# High
plt.subplot(3, 1, 1)
plt.plot(true_y_test[:, 0], label='True High', color='blue')
plt.plot(true_preds[:, 0], label='Predicted High', color='orange')
plt.title('High Price Prediction')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.legend()

# Low
plt.subplot(3, 1, 2)
plt.plot(true_y_test[:, 1], label='True Low', color='green')
plt.plot(true_preds[:, 1], label='Predicted Low', color='red')
plt.title('Low Price Prediction')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.legend()

# Close
plt.subplot(3, 1, 3)
plt.plot(true_y_test[:, 2], label='True Close', color='purple')
plt.plot(true_preds[:, 2], label='Predicted Close', color='brown')
plt.title('Close Price Prediction')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Plot predictions vs actuals for Low Price
plt.figure(figsize=(12, 6))
plt.plot(true_y_test[:, 1], label='True Low')  # Actual Low values
plt.plot(true_preds[:, 1], label='Predicted Low', color='red')  # Predicted Low values
plt.title('Low Price Prediction')
plt.xlabel('Time Steps')
plt.ylabel('Low Price (INR)')
plt.legend()
plt.show()


In [ ]:
# Plot predictions vs actuals for Close Price
plt.figure(figsize=(12, 6))
plt.plot(true_y_test[:, 2], label='True Close')  # Actual Close values
plt.plot(true_preds[:, 2], label='Predicted Close', color='black')  # Predicted Close values
plt.title('Close Price Prediction')
plt.xlabel('Time Steps')
plt.ylabel('Close Price (INR)')
plt.legend()
plt.show()


In [ ]:
import numpy as np

def predict_today_values(open_price, model, scaler, recent_data, seq_len=60, target_cols=[1, 2, 3]):
    """
    Predicts Close, High, and Low values for a given Open price.

    Parameters:
        open_price (float): The given Open price for today.
        model (keras.Model): The trained LSTM model.
        scaler (MinMaxScaler): The scaler used for feature scaling.
        recent_data (np.array): Recent historical data used to construct sequences.
        seq_len (int): The sequence length used for training the model.
        target_cols (list): Indices of target columns (High, Low, Close).

    Returns:
        dict: Predicted Close, High, and Low values in the original scale.
    """
    # Normalize the Open price
    normalized_open = scaler.transform([[open_price, 0, 0, 0, 0]])[0, 0]

    # Prepare the input sequence
    input_sequence = recent_data[-(seq_len - 1):].tolist()
    last_row = input_sequence[-1].copy()
    last_row[0] = normalized_open
    input_sequence.append(last_row)

    # Reshape for the LSTM model
    input_sequence = np.array(input_sequence).reshape(1, seq_len, recent_data.shape[1])

    # Predict using the model
    predictions = model.predict(input_sequence)

    # Inverse scale the predictions
    scaled_preds = np.zeros((1, recent_data.shape[1]))
    scaled_preds[:, target_cols] = predictions
    true_preds = scaler.inverse_transform(scaled_preds)[:, target_cols]

    # Extract results
    return {
        "Predicted High": true_preds[0, 0],
        "Predicted Low": true_preds[0, 1],
        "Predicted Close": true_preds[0, 2]
    }

open_price = 4012.30
recent_data = scaled_data[-100:]
predicted_values = predict_today_values(open_price, model, scaler, recent_data)
print(predicted_values)